Thans a lot for
https://www.kaggle.com/stpeteishii/commonlit-readability-bert-xgboost
First,I edit this script with LightGBM 
so,I run this script with optuna.
Final,run this script with best parameter

In [ ]:
!pip install transformers > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import os
import torch

import transformers
from transformers import BertTokenizer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split,KFold
#Finaly edit to  import lightgbm as lgb
import optuna.integration.lightgbm as lgb
#from xgboost import XGBRegressor
#Finally removeo  optuna.integration.
import optuna.integration.lightgbm as lgb
from tqdm import tqdm
tqdm.pandas()

In [ ]:
class CFG:
    seed = 46
    debug = False
    n_epoch = 10
    lr = 5e-5
    weight_decay = 1e-06
    n_splits = 5

In [ ]:
class BertSequenceVectorizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = '../input/huggingface-bert/bert-base-uncased'  # Inet-not-connect
        #self.model_name = 'bert-base-uncased'          # Inet-connect
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128

    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy()
        else:
            return seq_out[0][0].detach().numpy()

In [ ]:
train0 = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
train0[0:2]

In [ ]:
test0 = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
test0[0:2]

In [ ]:
sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
sample["target"] = 0

In [ ]:
train1=train0[['excerpt']].copy()
test1=test0[['excerpt']].copy()
target1=train0[['target']].copy()

In [ ]:
data1=pd.concat([train1,test1])
data1[0:2]

In [ ]:
n=len(train1)
print(n)

In [ ]:
BSV = BertSequenceVectorizer()

In [ ]:
data1['excerpt_bert']=data1['excerpt'].progress_apply(lambda x: BSV.vectorize(x))

In [ ]:
data1[0:2]

In [ ]:
excerpt2=[]
for item in data1['excerpt_bert']:
    excerpt2+=[item]

In [ ]:
X_train0=pd.DataFrame(excerpt2[0:n])
X_test0=pd.DataFrame(excerpt2[n:])
y_train0 = target1

In [ ]:
skf = KFold(n_splits = CFG.n_splits)

In [ ]:
for train_index,val_index in skf.split(X_train0,y_train0):
    
    x_train, x_val = X_train0.iloc[train_index], X_train0.iloc[val_index]
    y_train, y_val = y_train0.iloc[train_index], y_train0.iloc[val_index]

    train_data = lgb.Dataset(x_train, label=y_train)
    val_data = lgb.Dataset(x_val, label=y_val)

    #Finally edit this param  to final best param
    params = {
        'objective': 'regression',
        'metric': 'mae',
        "boosting": "gbdt",
        "verbosity": -1,
    }

    model = lgb.train(
        params,
        train_data,
        valid_names=['train', 'valid'],
        valid_sets=[train_data, val_data],
        verbose_eval=100,
    )

    best_params = model.params
    print(best_params)
   
    preds = model.predict(X_test0, num_iteration=model.best_iteration)
    sample['target'] += preds / CFG.n_splits

In [ ]:
subm=sample
subm.to_csv('submission.csv',index=None)
subm